In [54]:
import numpy as np
import networkx as nx
import random
import matplotlib.pyplot as plt

In [74]:
 """
Infection 
0:suceptible
    1:infected
    2:recoverd
    3:dead

"""

'Infection \n   0:suceptible\n   1:infected\n   2:recoverd\n   3:dead\n\n'

In [60]:
def kmcInitialize(G, rho):
    """Initialize kinetic Monte Carlo lists.
    Args:
        G:   Interaction graph.
        rho: Initial fraction of infected.
    Returns: 
        infectedList:       List of infected nodes.
        susceptibleList:    List of suscpetible nodes.
        adjSusceptibleList: List of susceptible nodes adjacent to infected ones.
        
    """
    # lists of infected and susceptible nodes
    infectedList = []
    susceptibleList = []
    deathList=[]
    recoveredList=[]
    
    # adjacent susceptible list
    adjSusceptibleList = []
    
    for u in G.nodes():
        if np.random.rand() < rho:
            G.nodes[u]['infected'] = 1
            infectedList.append(u)
        else:
            susceptibleList.append(u)
    
    for u in G.nodes():
        if G.nodes[u]['infected'] == 1:
            for v in G.neighbors(u):
                if G.nodes[v]['infected'] == 0:
                    adjSusceptibleList.append(v)
     
    return infectedList, susceptibleList, adjSusceptibleList

In [75]:
def Vaccination(G, u,VacCharter):
    """Vaccination.
    Args:
        G:                  Interaction graph.
        u:                  Node that recovers.
        infectedList:       List of infected nodes.
        susceptibleList:    List of suscpetible nodes.
        adjSusceptibleList: List of susceptible nodes adjacent to infected ones.
    
    Returns: 
        infectedList:       List of infected nodes.
        susceptibleList:    List of suscpetible nodes.
        adjSusceptibleList: List of susceptible nodes adjacent to infected ones.
    """
     
    # set state of node "u" to infected
    VacCharter=[0,1,3,4,5,1,34,34,3,2,3,4,5]    
    # update "adjSusceptibleList"
    for v in G.nodes(u):
        if G.nodes[v]['state'] == 1:
            adjSusceptibleList.remove(u) 
        if G.nodes[v]['infected'] == 0:
            adjSusceptibleList.append(v)
            
    return infectedList, susceptibleList, adjSusceptibleList

In [62]:
def kmcInfection(G, u, infectedList,susceptibleList,adjSusceptibleList):
    """Infection process.
    Args:
        G:                  Interaction graph.
        u:                  Node that recovers.
        infectedList:       List of infected nodes.
        susceptibleList:    List of suscpetible nodes.
        adjSusceptibleList: List of susceptible nodes adjacent to infected ones.
    
    Returns: 
        infectedList:       List of infected nodes.
        susceptibleList:    List of suscpetible nodes.
        adjSusceptibleList: List of susceptible nodes adjacent to infected ones.
    """
    
    # remove node "u" from "susceptibleList" and add it to "infectedList"
    susceptibleList.remove(u)
    infectedList.append(u)
     
    # set state of node "u" to infected
    G.nodes[u]['state'] = 1
        
    # update "adjSusceptibleList"
    for v in G.neighbors(u):
        if G.nodes[v]['state'] == 1: 
            adjSusceptibleList.remove(u) 
        if G.nodes[v]['infected'] == 0:
            adjSusceptibleList.append(v)
            
    return infectedList, susceptibleList, adjSusceptibleList

In [63]:
def kmcRecovery(G, u, infectedList,susceptibleList,adjSusceptibleList,deathList):
    """Recovery process.
    Args:
        G:                  Interaction graph.
        u:                  Node that recovers.
        infectedList:       List of infected nodes.
        susceptibleList:    List of suscpetible nodes.
        adjSusceptibleList: List of susceptible nodes adjacent to infected ones.
    
    Returns: 
        infectedList:       List of infected nodes.
        susceptibleList:    List of suscpetible nodes.
        adjSusceptibleList: List of susceptible nodes adjacent to infected ones.
    
    """
     
    # remove node "u" from "infectedList" and add it to "recoveredList"
    infectedList.remove(u)
    recoverdList.append(u)
    
    # set state of node "u" to recoverd
    G.nodes[u]['infected'] = 0 
    
    # update "adjSusceptibleList"
    for v in G.neighbors(u):
        if G.nodes[v]['infected'] == 0:
            adjSusceptibleList.remove(v)   
        if G.nodes[v]['infected'] == 1:
            adjSusceptibleList.append(u)

    return infectedList, susceptibleList, adjSusceptibleList

In [64]:
def kmcDeath(G, u, infectedList, deathList, adjSusceptibleList):
    """Recovery process.
    Args:
        G:                  Interaction graph.
        u:                  Node that recovers.
        infectedList:       List of infected nodes.
        deathList:    List of suscpetible nodes.
        adjSusceptibleList: List of susceptible nodes adjacent to infected ones.
    
    Returns: 
        infectedList:       List of infected nodes.
        susceptibleList:    List of suscpetible nodes.
        adjSusceptibleList: List of susceptible nodes adjacent to infected ones.
    
    """
     
    # remove node "u" from "infectedList" and add it to "deathList"
    infectedList.remove(u)
    susceptibleList.append(u)
    
    # set state of node "u" to death
    G.nodes[u]['state'] = 0 
    
    # update "adjSusceptibleList"
    for v in G.neighbors(u):
        if G.nodes[v]['state'] == 0:
            adjSusceptibleList.remove(v)   
        if G.nodes[v]['state'] == 1:
            adjSusceptibleList.append(u)

    return infectedList, susceptibleList, adjSusceptibleList

In [72]:
def add_ages(g,distribution):
    path1 = 'Age_Dists/'
    path2 = 'Dist.csv'
    #generate array containing amount of people per age group
    agedistribution = np.genfromtxt(path1+distribution+path2, delimiter=',')[:,1]
    #visit vertices in random order
    print(agedistribution)
    vs = list(G.nodes())
    shuffle(vs)
    #we will fill all age groups one by one
    a = 1 #current agegroup
    counter = 0 #counter for how many people have been placed in current group
    
    for v in vs:
        if(a > 20):
            #safety measure
            return
        if counter < agedistribution[a-1]:
            #if current age group hasn't gotten enough people assigned, add another, move on to next vertex
            age[v] = a
            agegrouplists[a-1].append(v)
            counter = counter + 1
        else:
            #current age group is full, move to next age group with at least one person
            a = a + 1
            while(agedistribution[a-1] == 0):
                if(a < 20):
                    a = a + 1
                else:
                    #in case number for last age group is 0
                    return
            age[v] = a
            agegrouplists[a-1].append(v)
            counter = 1

In [65]:
def PLOT():
        plt.figure()     
        x.draw(G, pos, edge_color="Grey", cmap=plt.get_cmap('RdYlGn_r'), node_color=[G.node[x]['infected'] for x in G.nodes()], vmin=0, vmax=1.0)
        #adjust the plot limits
        cut = 1.14
        xmax= cut*max(xx for xx,yy in pos.values())
        ymax= cut*max(yy for xx,yy in pos.values())   
        plt.xlim([-xmax, xmax])
        plt.ylim([-ymax, ymax])
        #plt.savefig('imgs/%04d.png'%len(t), dpi = 300)    
        #plt.close('all')

In [71]:
def kmcSIS(G, T, lamb, rho):
    """SIS kinetic Monte Carlo.
    Args:
        G:     Interaction graph.
        T:     Simulation time period.
        lamb:  Effective spreading rate.
        rho:   Initial fraction of infected.
        
    Returns:
        t:     Time array.
        S:     Susceptible array.
        I:     Infected array.
    """
    
    # initialize SIS dynamics
    infectedList, susceptibleList, adjSusceptibleList = kmcInitialize(G, rho)
    
    t_tot = 0

    # t, S, and I arrays
    t = []
    S = []
    I = []
    
    t.append(t_tot)
    S.append(len(susceptibleList))
    I.append(len(infectedList))
    
   
    #pos = nx.circular_layout(G) #dict( (n, n) for n in G.nodes() )
    t_output = 1
    while(t_tot < T):
        if t_tot > t_output:
            t_output += 1
            #print(t_tot)
        #plt.figure()     
        #nx.draw(G, pos, edge_color="Grey", cmap=plt.get_cmap('RdYlGn_r'), node_color=[G.node[x]['infected'] for x in G.nodes()], vmin=0, vmax=1.0)
        # adjust the plot limits
        #cut = 1.14
        #xmax= cut*max(xx for xx,yy in pos.values())
        #ymax= cut*max(yy for xx,yy in pos.values())   
        #plt.xlim([-xmax, xmax])
        #plt.ylim([-ymax, ymax])
        #plt.savefig('imgs/%04d.png'%len(t), dpi = 300)    
        #plt.close('all')
        
        
        Q1=np.array[20]

        # total infection rate    
            
        #Q1 = lamb*len(adjSusceptibleList)
        
        # total recovery rate
        #Q2 = len(infectedList)
        
        # total Deth rate
        #Q3 = lamb2*len(infectedList)
        
        Q1=np.zeros(20) #infection rate for 20 age groups
        Q2=np.zeros(20) #recovery rate for 20 age groups
        Q3=np.zeros(20) #death reate for 20 age groups
        
        
        # this is gona create a vector Q1
        for x in adjSuceptible:
            if G.nodes[adjSusceptible[x]]['vac']== 1:
                 Q1[[adjSusceptible[x]]['age']]=Q1[[adjSusceptible[x]]['age']]+vacinflist[[adjSusceptible[x]]['age']]
            else if G.nodes[adjSusceptible[x]]['vac']==0:
                 Q1[[adjSusceptible[x]]['age']]=Q1[[adjSusceptible[x]]['age']]+inflist[[adjSusceptible[x]]['age']]

        
   
        R = Q1 + Q2
                
        if R == 0:
            break
        
        # infection
        if   Q1/(Q1+Q2)<np.random.random() < Q1/(Q1+Q2):
            rand_state = random.randint(0, len(adjSusceptibleList)-1)
            infectedList, susceptibleList, adjSusceptibleList = kmcInfection(G, adjSusceptibleList[rand_state], infectedList, susceptibleList, adjSusceptibleList)
        
        #death
        else if :
            1/(Q1+Q2)<np.random.random() < Q1/(Q1+Q2):
            rand_state = random.randint(0, len(adjSusceptibleList)-1)
            infectedList, susceptibleList, adjSusceptibleList = kmcInfection(G, adjSusceptibleList[rand_state], infectedList, susceptible
            
        # recovery
        else if :
            rand_state = random.randint(0, len(infectedList)-1)
            infectedList, susceptibleList, adjSusceptibleList = kmcRecovery(G, infectedList[rand_state], infectedList, susceptibleList, adjSusceptibleList)

        delta_t = -np.log(1-random.random())/R
        t_tot += delta_t
        
        
        
        
        # update lists
        t.append(t_tot)
        S.append(len(susceptibleList))
        I.append(len(infectedList))
        
    for u in G.nodes():
        if np.random.rand() < rho:
            G.nodes[u]['infected'] = 1
            infectedList.append(u)
        else:
            susceptibleList.append(u)
    
   
    
    return np.asarray(t), np.asarray(S), np.asarray(I)

SyntaxError: invalid syntax (3081884948.py, line 49)

In [67]:
#probabilities


#death probability list for different age groups: https://pubmed.ncbi.nlm.nih.gov/32674819/
drlist = [.003,.003,.003,.003,.003,.003,.005,.005,.011,.011,.03,.03,.095,.095,.228,.228,.296,.296,.296,0.296]
#same VACCINATED
vac_drlist = [.001,.001,.001,.001,.001,.001,.002,.002,.005,0.005,0.015,0.015,.047,.047,.114,.114,.148,.148,.148,.148]


#vaccine availability by age group:
vacc_av = [0,0,0,0,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01]
#vaccine acceptance rate by age group:
vacc_ac = [0,0,0,0,0.5,0.5,0.5,0.5,0.6,0.6,0.6,0.6,0.7,0.7,0.7,0.7,0.8,0.8,0.9,0.9]

#infection probability for different age groups
inflist = [0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8]
#same VACCINATED
vac_inflist = [.20,.20,.20,.20,.20,.20,.20,.20,.20,.20,.20,.20,.20,.20,.20,.20,.20,.20,.20,0.20]

#recovery probability for different age groups
reclist = [0.34,0.34,0.34,0.34,0.34,0.34,0.34,0.34,0.32,0.32,0.28,0.28,0.22,0.22,0.15,0.15,0.10,0.10,0.08,0.08]
#same VACCINATED
vac_reclist = [0.39,0.39,0.39,0.39,0.39,0.39,0.39,0.39,0.35,0.35,0.31,0.31,0.26,0.26,0.20,0.20,0.14,0.14,0.13,0.13]

#economy contribution

economy = [0,0,0,0.025,0.025,0.05,0.1,0.1,0.1,0.2,0.1,0.1,0.1,0.05,0.025,0.025,0,0,0]

In [70]:
if __name__ == "__main__":
        
    T = 100        # simulation time period
    n = 100       # number of nodes
    
    rho = 0.05      # initial infected fraction
    lamb = 0.5      # effective spreading rate
    
    # interaction network
    G = nx.grid_2d_graph(int(n**0.5), int(n**0.5), periodic = True)
    nx.set_node_attributes(G, 0, 'state')
    nx.set_node_attributes(G, 0, 'age') 
    
    # simulate SIS dynamics with kinetic Monte Carlo
    t, S, I = kmcSIS(G, T, lamb, rho)
    
    plt.figure()
    plt.plot(t, S/len(G), linewidth = 1.5, label = r'$S$')
    plt.plot(t, I/len(G), linewidth = 1.5, label = r'$I$')
    plt.xlabel(r'Time')
    plt.ylabel(r'Fraction')
    plt.legend(frameon = False)
    plt.tight_layout()
    plt.show()

KeyError: 'infected'

In [69]:
    plt.figure()     
    nx.draw(G, pos, edge_color="Grey", cmap=plt.get_cmap('RdYlGn_r'), node_color=[G.nodes[x]['age'] for x in G.nodes()], vmin=0, vmax=1.0)
        #adjust the plot limits
    cut = 1.14
    xmax= cut*max(xx for xx,yy in pos.values())
    ymax= cut*max(yy for xx,yy in pos.values())   
    plt.xlim([-xmax, xmax])
    plt.ylim([-ymax, ymax])

NameError: name 'pos' is not defined

<Figure size 432x288 with 0 Axes>

In [73]:
G.nodes()

NodeView(((0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (6, 8), (6, 9), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7), (7, 8), (7, 9), (8, 0), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (8, 8), (8, 9), (9, 0), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6), (9, 7), (9, 8), (9, 9)))